In [1]:
# 2025 Tau-Seq Pipeline
    # Supertype level QC (RUN ALL)
# Method: Otero-Garcia 2023
# Donors: 12 (SEA-AD, Braak IV-VI)
# Tau-Seq Cells: Approx. 200k
# SEA-AD Reference Cells: Approx 1.5M
# By: Victoria Rachleff
# Date created: May 5 2025
# Last Modified: May 5 2025

# enviornments:
    # mapping: /allen/programs/celltypes/workgroups/hct/ktrav/conda/envs/iterative_scANVI/
    # relative abundance: /allen/programs/celltypes/workgroups/hct/ktrav/conda/envs/sccoda_019
    # differential expression: /allen/programs/celltypes/workgroups/hct/ktrav/conda/envs/nebula_153

# resource recs: 
    # mapping: salloc --ntasks-per-node=1 --cpus-per-task=32 --mem=500G --time=0-48:00:00 -p celltypes -G 1 srun --pty bash -i -l 
    # other: salloc --ntasks-per-node=1 --cpus-per-task=32 --mem=400G --time=0-12:00:00 -p celltypes srun --pty bash -i -l 

# Steps: 
    # Manually enter pTau status 
    # Technical QC (donors to exclude)
    # Integration and Mapping (scANVI)
    # Neighborhood QC (Custom function)
    # Relative Abundance (scCODA, Tstat)
    # Differential Expression (Nebula)

In [ ]:
# Load required packages
import os
import seaborn as sns
import numpy as np
import anndata
import re
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
import scanpy as sc
import copy
from datetime import datetime
from scipy.io import mmread
from igraph import *
print(anndata.__version__)
import i
import warnings
warnings.filterwarnings('ignore')

0.11.3


In [4]:
'''Astrocyte'''

# 13. Read in the subclass level objects

filepath_query = "/Users/nickkeesey/Desktop/projects/marimo_preprocess/test_data/ADsoma_pTau_only_Astrocyte.h5ad"
adata_query = sc.read_h5ad(filepath_query)


filepath_ref = "/Users/nickkeesey/Desktop/projects/marimo_preprocess/test_data/SEAAD_MTG_RNAseq_final-nuclei.2024-02-13_Astrocyte.h5ad"
adata_ref = sc.read_h5ad(filepath_ref)

In [6]:
print("Reference Donors:", adata_ref.obs["Donor ID"].unique()[:5])
print("Query Donors:", adata_query.obs["Donor ID"].unique()[:5])

Reference Donors: ['H20.33.026', 'H21.33.005', 'H21.33.013', 'H19.30.002', 'H20.33.028']
Categories (88, object): ['H18.30.001', 'H18.30.002', 'H19.30.001', 'H19.30.002', ..., 'H21.33.044', 'H21.33.045', 'H21.33.046', 'H21.33.047']
Query Donors: ['H20.33.046', 'H20.33.031', 'H20.33.018', 'H20.33.028', 'H20.33.040']
Categories (9, object): ['H20.33.008', 'H20.33.015', 'H20.33.018', 'H20.33.024', ..., 'H20.33.029', 'H20.33.031', 'H20.33.040', 'H20.33.046']


In [7]:
overlap = set(adata_ref.obs["Donor ID"]).intersection(set(adata_query.obs["Donor ID"]))
print(f"Number of overlapping donors: {len(overlap)}")

Number of overlapping donors: 9


In [8]:
# 8. Link key metadata features across datasets (from SEA-AD to pTau, same donors)
# Step 1: Ensure Donor IDs are strings and clean
adata_ref.obs["Donor ID"] = adata_ref.obs["Donor ID"].astype(str).str.strip()
adata_query.obs["Donor ID"] = adata_query.obs["Donor ID"].astype(str).str.strip()

# Step 2: Create donor-level metadata lookup from adata_ref
donor_metadata = (
    adata_ref.obs[[
        "Donor ID", 
        "Continuous Pseudo-progression Score", 
        "Age at Death", 
        "Sex", 
        "Braak", 
        "Thal", 
        "Overall AD neuropathological Change"
    ]]
    .drop_duplicates(subset="Donor ID")
    .set_index("Donor ID")
)

# Step 3: Map each column from donor_metadata into adata_query.obs by Donor ID
adata_query.obs["CPS"] = adata_query.obs["Donor ID"].map(donor_metadata["Continuous Pseudo-progression Score"])
adata_query.obs["Age"] = adata_query.obs["Donor ID"].map(donor_metadata["Age at Death"])
adata_query.obs["Sex"] = adata_query.obs["Donor ID"].map(donor_metadata["Sex"])
adata_query.obs["Braak"] = adata_query.obs["Donor ID"].map(donor_metadata["Braak"])
adata_query.obs["Thal"] = adata_query.obs["Donor ID"].map(donor_metadata["Thal"])
adata_query.obs["ADNC"] = adata_query.obs["Donor ID"].map(donor_metadata["Overall AD neuropathological Change"])

# Optional: CPS binarization
adata_query.obs["CPS_Bin"] = adata_query.obs["CPS"].apply(lambda x: 1 if pd.notnull(x) and x > 0.5 else 0)

In [9]:
# Rename to match as needed

# Consistent name: Doublet score
adata_query.obs['Doublet score'] = adata_query.obs['doublet_score']

# Consistent name: Fraction mitochondrial UMIs
# Generate the calculation (was this supposed to be included in data delivery?)
adata_query.obs['calculated pTau Fraction mito UMIs'] = (
    adata_query[:, adata_query.var_names.str.startswith("MT-")].X.sum(axis=1) / adata_query.X.sum(axis=1)
)
# adata_query.obs['Fraction mitochondrial UMIs'].describe() ## this is a blank column
adata_query.obs['Fraction mitochondrial UMIs'] = adata_query.obs['calculated pTau Fraction mito UMIs']

# Consistent name: Genes detected
adata_query.obs['Genes detected'] = adata_query.obs['n_genes']

In [10]:
# Identify columns with only one unique value, including NaN and empty string
cols_to_drop = [
    col for col in adata_query.obs.columns
    if adata_query.obs[col].nunique(dropna=False) <= 1 and col != "Source"
]

# Drop them
adata_query.obs = adata_query.obs.drop(columns=cols_to_drop)

print(f"Dropped columns (excluding 'Source'): {cols_to_drop}")

Dropped columns (excluding 'Source'): ['facs_population_plan', 'library_prep_pass_fail', 'sample_id', 'Neurotypical reference', 'Organism', 'Brain Region', 'Gender', 'Age at Death', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MOCA in months', 'APOE Genotype', 'Primary Study Name', 'Secondary Study Name', 'NeuN positive fraction on FANS', 'RIN', 'cell_prep_type', 'rna_amplification', 'Number of mapped reads', 'Number of unmapped reads', 'Number of multimapped reads', 'Number of reads', 'Number of UMIs', 'Class confidence', 'Subclas

In [11]:
from iterative_scANVI import mapping as isc_mapping

sc.set_figure_params(figsize=(6, 6), frameon=False)

dataset = "Astrocyte_ADsoma_20250513"
region = "MTG"
filepath = "/allen/programs/celltypes/workgroups/humancelltypes/VictoriaR/2025_Tau-seq_subclass_QC"


isc_mapping.iteratively_map(
    adata_query,
    adata_ref,
    output_dir=os.path.join(filepath, "models", region + "_" + dataset),
    labels_keys=["supertype"],
    **{
        "batch_key": "Source",
        "categorical_covariate_keys": ["Donor ID"],
        "dispersion": "gene-batch",
        "plot_latent_space": True,
      }
)

ModuleNotFoundError: No module named 'iterative_scANVI'

In [ ]:
from joblib import parallel_backend
import hashlib
import json

# NEW IN ANNDATA 6, ADD ARGUMENT 'SUBCLASS' AND BE SURE TO WRITE SUBCLASS NAME

save_anndata6(
    adata_query=adata_query,
    adata_ref=adata_ref,
    split_key=None, 
    groupby="supertype",
    subclass="Astrocyte",
    output_dir = f"/allen/programs/celltypes/workgroups/humancelltypes/VictoriaR/2025_Tau-seq_subclass_QC/models/MTG_Astrocyte_ADsoma_20250513", # Hard coded !!! CHANGE EACH TIME !!!
    date="2025-05-13", # important: if run carries over to different date, must udpate!
    model_args={
        "batch_key": "Source",
        "categorical_covariate_keys": ["Donor ID"],
        "dispersion": "gene-batch",
    },
    **{
        "n_cores": 32,
        "cluster_cells": True
    }
)